In [1]:
#!pip install jupyter_dash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 78.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 51.2 MB/s eta 0:00:00


In [ ]:
# Import required libraries
import pandas as pd
from app import app
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.express as px
from numpy.core.fromnumeric import sort
from jupyter_dash import JupyterDash


In [384]:
df = pd.read_csv("merge.csv")

f = lambda x: (x["Tarih"].split('/'))[2]
df["Yıl"] = df.apply(f, axis=1)
df["Yıl"]=df["Yıl"].astype('int')

df.drop(df[df.Yıl < 2019].index, inplace=True)


city_dropdown = df['İl/ilçe'].unique()
city_dropdown=sort(city_dropdown)

city_list=list()
for i in city_dropdown:
  city_list.append(i)

year_dropdown = df['Yıl'].unique()
year_list=list()

for i in year_dropdown:
  year_list.append(i)

In [385]:
server = app.server
app.config.suppress_callback_exceptions = True

app = JupyterDash(__name__,external_stylesheets=["https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap-grid.min.css"])
tabs_styles = {"height": "5%","width": "2%"}
tabs_container_styles = {"width": "48%", "height": "%45"}

app.layout = html.Div(
                                children=[
                                    
                                        html.H1('2019 - 2023 KADIN CİNAYETLERİ ♀️  ',
                                        style={'textAlign': 'center',
                                               'font-size': 40}),
                                            
                                    dbc.Container([
 
                                                dbc.Row([
                                                    
                                                    dbc.Col(dcc.RangeSlider(marks={i: '{}'.format(i) for i in range(2019,2024,1)},
                                                            value=[df["Yıl"].min(),df["Yıl"].max()],
                                                            dots=False,
                                                            step=10,
                                                            id='year_slider')),
                                                         
                                                    dbc.Col(html.H4("TOPLAM ÖLDÜRÜLEN KADIN SAYISI",style={"textAlign":"center", "width":"98%","background-color":"#0C0B22","padding":"4%"}))

                                                ]),
                                                dbc.Row([
                                                    
                                                    dbc.Col(dcc.Dropdown(
                                                                        id = 'citydrop',
                                                                        placeholder = 'İl Seçiniz',
                                                                        searchable = True, 
                                                                        options=city_list,
                                                                        value='İstanbul')),
                                                         
                                                    dbc.Col(html.H2(df.shape[0],style={"textAlign":"center", "width":"98%","background-color":"#0C0B22","margin-top":"0px","padding":"2%"}))

                                                ]),

                                                dbc.Row([
                                                                                      
                                                    dbc.Col(dcc.Graph(id="pieyear")),
                                                    dbc.Col(dcc.Graph(id="piecity"))

                                                ]),
                                                
                                                dbc.Row([
                                                    
                                                    dbc.Col(
                                                            dcc.Tabs(id='tab', value='tab-1', children=[
                                                            dcc.Tab(label='Maktülün Yaşı', value='tab-1'),
                                                            dcc.Tab(label='Öldürülme Sebebi', value='tab-2'),
                                                            dcc.Tab(label='Fail', value='tab-3',style={"margin":"0px"}),
                                                            dcc.Tab(label='Korunma Talebi', value='tab-4'),
                                                            dcc.Tab(label='Öldürülme Şekli', value='tab-5'),
                                                            dcc.Tab(label='Failin Durumu', value='tab-6'),
                                                        ],style={"hight": "49%" ,"hight": "49%","margin-top":"2%"})),

                                                dbc.Col(html.H4("KONUM BİLGİSİNE GÖRE CİNAYET BİLGİLERİ",style={"textAlign":"center", "width":"98%","background-color":"#0C0B22","padding":"1%","margin-top":"10%"}))
                                                    
                                                    ]),

                                                dbc.Row([

                                                         dbc.Col(html.Div(id='tabs-example-content-1')),
                                                         dbc.Col(dcc.Graph(id="map"),style={"padding-top":"60px"})
                                                    
                                                    ]),
                                                
                                            ], fluid=True),

                                 ],style={"color":"rgb(178,24,43)","background-color":"#010914"}
)

@app.callback(
    
    Output('tabs-example-content-1', 'children'),
    [Input('tab', 'value'),
     Input("year_slider","value")],

)
  
def render_content(tab,year_slider):
    low, high = year_slider
    if tab == 'tab-1':
        return html.Div([
            html.H3('Yıl Bilgisine Göre Maktülün Yaşı'),
            dcc.Graph(
                figure=dict(
                    data=[
                        dict(
                        x=df[df["Yıl"].between(low,high)]['Maktülün Yaşı'].value_counts().index,
                        y=df[df["Yıl"].between(low,high)]['Maktülün Yaşı'].value_counts().values,
                        type='bar',
                        marker = { "color" : px.colors.sequential.RdBu},
                        
                    )]
                )
            )
        ])

    elif tab == 'tab-2':
        return html.Div([
            html.H3('Yıl Bilgisine Göre İlk 10 Öldürülme Sebebi'),
            dcc.Graph(
                figure=dict(
                    data=[
                        dict(
                        x=df[df["Yıl"].between(low,high)]['Öldürülme Sebebi'].value_counts().head(10).index,
                        y=df[df["Yıl"].between(low,high)]['Öldürülme Sebebi'].value_counts().head(10).values,
                        type='bar',
                        marker = { "color" : px.colors.sequential.RdBu},
                    )]
                )
            )
        ])
    
    elif tab == 'tab-3':
        return html.Div([
            html.H3('Yıl Bilgisine Göre İlk 10 Fail'),
            dcc.Graph(
                figure=dict(
                    data=[
                        dict(
                        x=df[df["Yıl"].between(low,high)]['Fail'].value_counts().head(10).index,
                        y=df[df["Yıl"].between(low,high)]['Fail'].value_counts().head(10).values,
                        type='bar',
                        marker = { "color" : px.colors.sequential.RdBu},
                    )]
                )
            )
        ])

    elif tab == 'tab-4':
        return html.Div([
            html.H3('Yıl Bilgisine Göre Korunma Talebi'),
            dcc.Graph(
                figure=dict(
                    data=[
                        dict(
                        x=df[df["Yıl"].between(low,high)]['Korunma Talebi'].value_counts().index,
                        y=df[df["Yıl"].between(low,high)]['Korunma Talebi'].value_counts().values,
                        type='bar',
                        marker = { "color" : px.colors.sequential.RdBu},
                    )]
                )
            )
        ])

    elif tab == 'tab-5':
        return html.Div([
            html.H3('Yıl Bilgisine Göre İlk 10 Öldürülme Şekli'),
            dcc.Graph(
                figure=dict(
                    data=[
                        dict(
                        x=df[df["Yıl"].between(low,high)]['Öldürülme Şekli'].value_counts().head(10).index,
                        y=df[df["Yıl"].between(low,high)]['Öldürülme Şekli'].value_counts().head(10).values,
                        type='bar',
                        marker = { "color" : px.colors.sequential.RdBu},
                    )]
                )
            )
        ])

    elif tab == 'tab-6':
        return html.Div([
            html.H3('Yıl Bilgisine Göre Failin Durumu'),
            dcc.Graph(
                figure=dict(
                    data=[
                        dict(
                        x=df[df["Yıl"].between(low,high)]['Failin Durumu'].value_counts().index,
                        y=df[df["Yıl"].between(low,high)]['Failin Durumu'].value_counts().values,
                        type='bar',
                        marker = { "color" : px.colors.sequential.RdBu},
                    )]
                )
            )
        ])

@app.callback(
    
    Output("map", "figure"), 
    [Input("citydrop", "value"), 
    Input("year_slider","value")],

    )

def map_chart(citydrop,year_slider):

    low, high = year_slider

    if (df[(df["İl/ilçe"]==citydrop) &  (df["Yıl"].between(low, high))].size)==0:

      Enlem=df[(df["İl/ilçe"]=="İstanbul") &  (df["Yıl"]==2023)].index[0]
      Enlem=df.loc[Enlem].Enlem_x

      Boylam=df[(df["İl/ilçe"]=="İstanbul") &  (df["Yıl"]==2023)].index[0]
      Boylam=df.loc[Boylam].Boylam_x

    else:
    
      Enlem=df[(df["İl/ilçe"]==citydrop) &  (df["Yıl"].between(low, high))].index[0]
      Enlem=df.loc[Enlem].Enlem_x

      Boylam=df[(df["İl/ilçe"]==citydrop) &  (df["Yıl"].between(low, high))].index[0]
      Boylam=df.loc[Boylam].Boylam_x

    figure = px.scatter_mapbox(df, lat='Enlem_x', lon='Boylam_x',
                        center = dict(lat = Enlem	, lon = Boylam), # This is the coordinate of San Francisco..
                        zoom = 8,                                # Zoom of the map
                        opacity = .9,
                        title = 'Konuma Göre Cinayet Bilgileri ',
                        mapbox_style="stamen-toner",
                        color_continuous_scale=px.colors.sequential.Rainbow, 
                                      
                        hover_data = ["Ad Soyad","Maktülün Yaşı","İl/ilçe","Tarih","Öldürülme Sebebi","Fail","Korunma Talebi","Öldürülme Şekli","Failin Durumu"]
                        )

    figure.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    
    return figure

@app.callback(
    Output("pieyear", "figure"),
    [
     Input("year_slider","value")]
)

def pie_chart(year_slider):

  low, high = year_slider

  ls_values=list()
  ls_index=list()

  for i in df[df["Yıl"].between(low,high)]['Yıl'].value_counts().values:
    ls_values.append(i)
  for i in df[df["Yıl"].between(low,high)]['Yıl'].value_counts().index:
    ls_index.append(i)

  #figure = px.line(df, x=ls_index, y=ls_values, title='Life expectancy in Canada')
  fig = px.pie(df, values=ls_values, names=ls_index, hole=.3,color_discrete_sequence=px.colors.sequential.RdBu,title='Yıl Bilgisine Göre Toplam Cinayet')
  fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction="left",
            buttons=list([
                dict(
                    args=["type", "pie"],
                    label="Pie Chart",
                    method="restyle"
                ),
                dict(
                    args=["type", "bar"],
                    label="Bar Chart",
                    method="restyle",
                    
                )
            ]),
        ),
    ]
)
  
  fig.update_layout({
      
  #'plot_bgcolor': 'rgba(15,15,15, 0.1)',
  'paper_bgcolor': '#0C0B22',
  "font_color":"rgb(178,24,43)",
  "title_font_color":"white",
  "legend_title_font_color":"white",

  })
  
  return fig

@app.callback(
    
    Output("piecity", "figure"),
    [Input("citydrop","value"),
    Input("year_slider","value")]
  
    )

def pie_chart(citydrop,year_slider):

  low,high=year_slider
  ls_values=list()
  ls_index=list()

  for i in df[df["Yıl"].between(low,high)]['İl/ilçe'].value_counts().head(20).values:
    ls_values.append(i)
  for i in df[df["Yıl"].between(low,high)]['İl/ilçe'].value_counts().head(20).index:
    ls_index.append(i)

  #figure = px.line(df, x=ls_index, y=ls_values, title='Life expectancy in Canada')
  fig = px.pie(df, values=ls_values, names=ls_index, hole=.3,color_discrete_sequence=px.colors.sequential.RdBu,title='Konum Bilgisine Göre Toplam Cinayet')

  fig.update_layout(
    updatemenus=[
        dict(
            type="buttons",
            direction="left",
            buttons=list([
                dict(
                    args=["type", "pie"],
                    label="Pie Chart",
                    method="restyle"
                ),
                dict(
                    args=["type", "bar"],
                    label="Bar Chart",
                    method="restyle",                    
                )
            ]),
        ),
    ]
)
  
  fig.update_layout({
  #'plot_bgcolor': 'rgba(15,15,15, 0.1)',
  'paper_bgcolor': '#0C0B22',
  "font_color":"rgb(178,24,43)",
  "title_font_color":"white",
  "legend_title_font_color":"white",
  
  })
  
  return fig

# Run the app
if __name__ == '__main__':
    app.run_server()

Dash app running on:


<IPython.core.display.Javascript object>